In [1]:
!pip install langchain langchain-openai pandas
!pip install openai
!pip install nest-asyncio
!pip install langchain-experimental
!pip install langchain_core langchain-anthropic langgraph

In [2]:
# Install package first
# We use uv for faster installation

# Google Colab is setting some uv-related environment variables, that seem to have broken with the a Colab update.
# The Colab team is tracking this internally (googlecolab/colabtools#5237).
# https://github.com/googlecolab/colabtools/issues/5237#issuecomment-2786440777
import os
os.environ["UV_CONSTRAINT"] = os.environ["UV_BUILD_CONSTRAINT"] = ""  # add this line to solve the uv related issue

!pip install uv
!uv pip install -q autogluon.timeseries --system --prerelease allow
!uv pip uninstall -q torchaudio torchvision torchtext --system # fix incompatible package versions on Colab


## if there seems to be inconsistency due to numpy & pandas versions, do the following
# !uv pip uninstall pandas numpy
!uv pip install -q --force-reinstall numpy==1.26.4 --system --prerelease allow
!uv pip install -q --force-reinstall pandas==2.2.2 --system --prerelease allow
## You may need to restart colab kernel

In [3]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [4]:
!pip install gradio

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [6]:
%cd /content/drive/MyDrive/GE_Capstone/AI Agent Workflow/

/content/drive/MyDrive/GE_Capstone/AI Agent Workflow


In [15]:
import pandas as pd
import numpy as np
import re
import json
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph import StateGraph, END, MessagesState
from langgraph.types import Command
from pydantic import BaseModel, Field
from typing import Literal
import nest_asyncio
from langgraph.func import entrypoint, task
from langchain_openai import OpenAI, ChatOpenAI
from langchain_experimental.agents import create_pandas_dataframe_agent

from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
# replace api_key with your own for following code to work
llm_4 = ChatOpenAI(
    api_key="xxx",
    model_name="gpt-4",
    temperature=0.7
)

llm_turbo = ChatOpenAI(
    api_key="xxx",
    model_name="gpt-3.5-turbo",
    temperature=0.7
)

In [39]:
class Route(BaseModel):
    step: Literal["forecast", "threshold_exceedance", "slope_calculation", "separate"] = Field(
        ..., description="The next step in the routing process"
    )

router = llm_4.with_structured_output(Route)

# ==== Supervisor Node ====
def llm_supervisor_node(state: MessagesState) -> Command[str]:
    user_msg = state["messages"][-1].content
    step = router.invoke([
        SystemMessage(content="Route the input to forecast, threshold_exceedance, slope_calculation, or separate based on the user's request."),
        HumanMessage(content=user_msg),
    ]).step
    return Command(update={"messages": state["messages"]}, goto=step)

# ==== Forecast Node ====
import uuid
def chronos_forecast_node(state: MessagesState) -> Command:
    user_msg = state["messages"][-1].content
    import re, json
    match = re.search(r"'([^']+\.csv)'", user_msg)
    pred_len_match = re.search(r"(\d+)\s+(?:days|steps)", user_msg)

    if not match or not pred_len_match:
        return Command(update={"messages": state["messages"] + [AIMessage(content="Error: Missing file path or prediction length.")]}, goto="end")

    file_path = match.group(1)
    pred_len = int(pred_len_match.group(1))

    df = pd.read_csv(file_path)[["item_id", "timestamp", "target"]]
    tsdf = TimeSeriesDataFrame.from_data_frame(df, id_column="item_id", timestamp_column="timestamp")
    train_data, test_data = tsdf.train_test_split(pred_len)
    predictor = TimeSeriesPredictor(prediction_length=pred_len).fit(train_data, presets="bolt_small")
    predictions = predictor.predict(train_data)
    leaderboard = predictor.leaderboard(test_data)
    model_name = leaderboard.iloc[0]["model"]
    mape = -leaderboard.iloc[0]["score_test"]

    predictions_df = predictions.to_data_frame().reset_index()
    preview = predictions_df[["item_id", "timestamp", "mean"]].tail(10).to_string(index=False)

    # ==== Save predictions to temporary CSV ====
    output_path = f"/tmp/forecast_result_{uuid.uuid4().hex}.csv"
    predictions_df.to_csv(output_path, index=False)

    result = {
        "content": f"Top model: {model_name}, MAPE: {mape * 100:.2f}%\nForecast Preview:\n{preview}",
        "file_path": output_path  # include for GUI download
    }
    return Command(update={"messages": state["messages"] + [AIMessage(content=json.dumps(result))]}, goto="end")


# ==== build agent for LangChain Pandas to use ====
def build_agent_from_prompt(llm, prompt: str):
    import re
    match = re.search(r"[\"'](?P<filename>[^\"']+\.csv)[\"']", prompt)
    if not match:
        raise ValueError("No CSV file name found in prompt.")
    file_path = match.group("filename")

    try:
        df = pd.read_csv(file_path)
    except Exception as e:
        raise ValueError(f"Failed to load CSV file: {file_path}") from e

    agent = create_pandas_dataframe_agent(llm, df, verbose=True, allow_dangerous_code=True)
    return agent

# ==== Slope Node ====
def slope_agent_node(state: MessagesState) -> Command:
    prompt = state["messages"][-1].content
    agent = build_agent_from_prompt(llm_4, prompt)
    result = agent.invoke(prompt)
    content = result.get("output", str(result))
    return Command(update={"messages": state["messages"] + [AIMessage(content=content)]}, goto="end")

# ==== Threshold Node ====
def threshold_agent_node(state: MessagesState) -> Command:
    prompt = state["messages"][-1].content
    agent = build_agent_from_prompt(llm_turbo, prompt)
    result = agent.invoke(prompt)
    content = result.get("output", str(result))
    return Command(update={"messages": state["messages"] + [AIMessage(content=content)]}, goto="end")

/usr/local/lib/python3.11/dist-packages/langchain_openai/chat_models/base.py:1637: UserWarning: Cannot use method='json_schema' with model gpt-4 since it doesn't support OpenAI's Structured Output API. You can see supported models here: https://platform.openai.com/docs/guides/structured-outputs#supported-models. To fix this warning, set `method='function_calling'. Overriding to method='function_calling'.
  warnings.warn(


In [40]:
# ==== Separate Node ====
def separate_worker_node(state: MessagesState) -> Command:
    user_msg = state["messages"][-1].content

    result = llm_4.invoke(user_msg)
    prompts = result.content.splitlines()

    all_outputs = []
    for prompt in prompts:
        prompt = prompt.strip()
        if not prompt:
            continue
        print(f"[Separate] Sub-prompt: {prompt}")

        sub_state = {"messages": [HumanMessage(content=prompt)]}
        sub_result = graph.invoke(sub_state)

        if "messages" in sub_result and isinstance(sub_result["messages"], list):
            output_msg = sub_result["messages"][-1]
            if hasattr(output_msg, "content"):
                all_outputs.append(f"Prompt: {prompt}\nResult: {output_msg.content}")
            else:
                all_outputs.append(f"Prompt: {prompt}\nResult: [No content returned]")
        else:
            all_outputs.append(f"Prompt: {prompt}\nResult: [Invalid result structure]")

    summary = "\n\n".join(all_outputs)
    return Command(update={"messages": state["messages"] + [AIMessage(content=summary)]}, goto="end")

In [41]:
builder = StateGraph(MessagesState)

builder.add_node("supervisor", llm_supervisor_node)
builder.add_node("forecast", chronos_forecast_node)
builder.add_node("threshold_exceedance", threshold_agent_node)
builder.add_node("slope_calculation", slope_agent_node)
builder.add_node("separate", separate_worker_node)

builder.set_entry_point("supervisor")
builder.add_edge("forecast", END)
builder.add_edge("threshold_exceedance", END)
builder.add_edge("slope_calculation", END)
builder.add_edge("separate", END)

graph = builder.compile()

In [34]:
def run_agentic_workflow(task_list, file_obj, pred_steps):
    from langchain_core.messages import HumanMessage
    import os

    # Get uploaded file path
    file_path = file_obj.name  # Gradio returns a file-like object

    prompts = []
    for task in task_list:
        if task == "Forecast":
            prompts.append(f"Please forecast the next {pred_steps} days using the file '{file_path}'.")
        elif task == "Threshold Exceedance":
            prompts.append(f"Use the file '{file_path}' to calculate the 95% threshold for target column of df, then display the list of values in the target column that exceed the 95% value only.")
        elif task == "Slope Calculation":
            prompts.append(f"Use the file '{file_path}' to calculate the slope of values in the target column of df, then return the slope of target column only.")

    results = []
    for prompt in prompts:
        state = {"messages": [HumanMessage(content=prompt)]}
        result = graph.invoke(state)
        results.append(f"[{prompt}]\n{result['messages'][-1].content}\n")

    return "\n\n".join(results)

gr.Interface(
    fn=run_agentic_workflow,
    inputs=[
        gr.CheckboxGroup(["Forecast", "Threshold Exceedance", "Slope Calculation"], label="Select Time Series Tasks"),
        gr.File(file_types=[".csv"], label="Upload CSV File"),
        gr.Slider(1, 200, value=24, label="Prediction Steps")
    ],
    outputs="text",
    title="LangGraph Time Series Agentic Workflow"
).launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://848986448c6a38f01f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [42]:
import gradio as gr
from langchain_core.messages import HumanMessage

def run_agentic_workflow(task_list, file_obj, pred_steps):
    import os
    file_path = file_obj.name

    prompts = []
    for task in task_list:
        if task == "Forecast":
            prompts.append(f"Please forecast the next {pred_steps} days using the file '{file_path}'.")
        elif task == "Threshold Exceedance":
            prompts.append(f"Use the file '{file_path}' to calculate the 95% threshold for target column of df, then display all values in the target column that exceed the 95% value only.")
        elif task == "Slope Calculation":
            prompts.append(f"Use the file '{file_path}' to calculate the slope of values in the target column of df, then return the slope of target column only.")

    results = []
    for prompt in prompts:
        state = {"messages": [HumanMessage(content=prompt)]}
        result = graph.invoke(state)
        results.append(f"[{prompt}]\n{result['messages'][-1].content}\n")

    return gr.update(visible=False), "\n\n".join(results)

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("## LangGraph Time Series Agentic Workflow")

    with gr.Row():
        task_input = gr.CheckboxGroup(["Forecast", "Threshold Exceedance", "Slope Calculation"], label="Select Time Series Tasks")
    file_input = gr.File(file_types=[".csv"], label="Upload CSV File")
    steps_input = gr.Slider(1, 200, value=24, label="Prediction Steps")

    with gr.Row():
        run_button = gr.Button("Submit", variant="primary")
        clear_button = gr.Button("Clear")

    output_text = gr.Textbox(label="Output")
    download_file = gr.File(label="Download Forecast Result")

    def wrapped_run(task_list, file_obj, pred_steps):
        from langchain_core.messages import HumanMessage
        import os

        file_path = file_obj.name
        prompts = []
        for task in task_list:
            if task == "Forecast":
                prompts.append(f"Please forecast the next {pred_steps} days using the file '{file_path}'.")
            elif task == "Threshold Exceedance":
                prompts.append(f"Use the file '{file_path}' to calculate the 95% threshold for target column of df, then display the list of values in the target column that exceed the 95% value only.")
            elif task == "Slope Calculation":
                prompts.append(f"Use the file '{file_path}' to calculate the slope of values in the target column of df, then return the slope of target column only.")

        results = []
        forecast_csv_path = None
        for prompt in prompts:
            state = {"messages": [HumanMessage(content=prompt)]}
            result = graph.invoke(state)
            msg = result["messages"][-1].content
            try:
                parsed = json.loads(msg)
                results.append(f"[{prompt}]\n{parsed['content']}")
                forecast_csv_path = parsed.get("file_path", None)
            except Exception:
                results.append(f"[{prompt}]\n{msg}")

        return "\n\n".join(results), forecast_csv_path

    run_button.click(
        wrapped_run,
        inputs=[task_input, file_input, steps_input],
        outputs=[output_text, download_file]
    )

    clear_button.click(
        lambda: ("", None),
        inputs=[],
        outputs=[output_text, download_file]
    )

demo.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://795357b58ccb2b4359.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
